# Automated Machine Learning for Earth Science via AutoGluon

## Authors

- Author1 = {"name": "Xingjian Shi", "affiliation": "Amazon Web Services", "email": "xjshi@amazon.com", "orcid": ""}
- Author2 = {"name": "Wen-ming Ye", "affiliation": "Amazon Web Services", "email": "wye@amazon.com", "orcid": ""}
- Author3 = {"name": "Nick Erickson", "affiliation": "Amazon Web Services", "email": "neerick@amazon.com", "orcid": ""}
- Author4 = {"name": "Jonas Mueller", "affiliation": "Amazon Web Services", "email": "jonasmue@amazon.com", "orcid": ""}
- Author5 = {"name": "Alexander Shirkov", "affiliation": "Amazon Web Services", "email": "ashyrkou@amazon.com", "orcid": ""}
- Author6 = {"name": "Zhi Zhang", "affiliation": "Amazon Web Services", "email": "zhiz@amazon.com", "orcid": ""}
- Author7 = {"name": "Mu Li", "affiliation": "Amazon Web Services", "email": "mli@amazon.com", "orcid": ""}
- Author8 = {"name": "Alexander Smola", "affiliation": "Amazon Web Services", "email": "alex@smola.org", "orcid": ""}

## Table of Contents
* [Purpose](#purpose)
* [Setup](#setup)
* [Forest Cover Type Classification](#forest-cover-type-classification)
    * [Train Model with One Line](#train-model-with-one-line)
    * [Evaluation and Prediction](#evaluation-and-prediction)
    * [Load the Predictor](#load-the-predictor)
    * [Feature Importance](#feature-importance)
    * [Achieve Better Performance](#achieve-better-performance)
* [Solar Radiation Prediction](#solar-radiation-predictions)
* [More Information](#more-information)

## Purpose

In this notebook, we introduce [AutoGluon](https://github.com/awslabs/autogluon) to the Earth science community. AutoGluon is an automated machine learning toolkit that enables users to solve machine learning problems with a single line of code. Many earth science problems involve tabular-like datasets. With AutoGluon, you can feed in the **raw** data table and specify the `label` column. AutoGluon will deliver a model that has reasonable performance in a short period of time. In addition, with AutoGluon, you can also analyze the importance of each feature column with a single line of code. In the following, we illustrate how to use AutoGluon to build machine learning models for two Earth Science problems.

## Setup

We will install [AutoGluon](https://github.com/awslabs/autogluon) via pip and also fix the random seed.

In [1]:
!python3 -m pip install autogluon --quiet
!python3 -m pip install mxnet --quiet
import random
import numpy as np
random.seed(123)
np.random.seed(123)

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Forest Cover Type Classification

In the first example, we will predict the forest cover type (the predominant kind of tree cover) from strictly cartographic variables. The dataset is downloaded from [Kaggle Forest Cover Type Prediction](https://www.kaggle.com/c/forest-cover-type-prediction). Study area of the dataset includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. The actual forest cover type for a given 30 x 30 meter cell was determined from US Forest Service (USFS) Region 2 Resource Information System data. Independent variables were then derived from data obtained from the US Geological Survey and USFS. The data is in raw form and contains binary columns of data for qualitative independent variables such as wilderness areas and soil type. Let's first download the dataset.

In [2]:
!wget https://deep-earth.s3.amazonaws.com/datasets/earthcube2021_demo/forest-cover-type-prediction.zip -O forest-cover-type-prediction.zip
!unzip -o forest-cover-type-prediction.zip -d forest-cover-type-prediction

--2021-04-14 04:10:03--  https://deep-earth.s3.amazonaws.com/datasets/earthcube2021_demo/forest-cover-type-prediction.zip
Resolving deep-earth.s3.amazonaws.com (deep-earth.s3.amazonaws.com)... 52.217.110.204
Connecting to deep-earth.s3.amazonaws.com (deep-earth.s3.amazonaws.com)|52.217.110.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26555059 (25M) [application/zip]
Saving to: ‘forest-cover-type-prediction.zip’

forest-cover-type-p 100%[===================>]  25.32M  56.1MB/s    in 0.5s    

2021-04-14 04:10:04 (56.1 MB/s) - ‘forest-cover-type-prediction.zip’ saved [26555059/26555059]

Archive:  forest-cover-type-prediction.zip
  inflating: forest-cover-type-prediction/sampleSubmission.csv  
  inflating: forest-cover-type-prediction/sampleSubmission.csv.zip  
  inflating: forest-cover-type-prediction/test.csv  
  inflating: forest-cover-type-prediction/test.csv.zip  
  inflating: forest-cover-type-prediction/test3.csv  
  inflating: forest-cover-type-pr

Here, we load and visualize the dataset. We will split the dataset to 80% training and 20% development for the purpose of reporting the score on the development data. Also, for the purpose of demonstration, we will subsample the dataset to 5000 samples.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('forest-cover-type-prediction/train.csv.zip')
df = df.drop('Id', 1)
df = df.sample(5000, random_state=100)
train_df, dev_df = train_test_split(df, random_state=100)

By visualizing the dataset, we can see that there are 54 feature columns and 1 label column called `"Cover_Type"`.

In [4]:
train_df.head(5)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
7449,2762,17,16,270,49,2639,206,206,134,268,...,0,0,0,0,0,0,0,0,0,5
13086,2283,109,11,0,0,1138,240,227,116,1187,...,0,0,0,0,0,0,0,0,0,4
14221,3220,82,14,247,66,3328,239,214,103,819,...,1,0,0,0,0,0,0,0,0,1
768,3021,68,8,201,26,4134,228,225,130,2493,...,0,0,0,0,0,0,0,0,0,1
6132,2446,76,21,469,105,726,241,196,75,1401,...,0,0,0,0,0,0,0,0,0,6


### Train Model with One Line

Next, we train a model in AutoGluon with a single line of code. We will just need to specify the label column before calling `.fit()`. Here, the label column is `Cover_Type`. AutoGluno will inference the problem type automatically. In our example, it can correctly figure out that it is a "multiclass" classification problem and output the model with the best accuracy. Internally, it will also figure out the feature type automatically.

In [5]:
import autogluon
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='Cover_Type', path='ag_ec2021_demo').fit(train_df)

Beginning AutoGluon training ...
AutoGluon will save models to "ag_ec2021_demo/"
AutoGluon Version:  0.1.0
Train Data Rows:    3750
Train Data Columns: 54
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	7 unique label values:  [5, 4, 1, 6, 3, 2, 7]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
NumExpr defaulting to 8 threads.
Train Data Class Count: 7
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    32024.06 MB
	Train Data (Original)  Memory Usage: 1.62 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTyp

	0.802	 = Validation accuracy score
	22.74s	 = Training runtime
	0.31s	 = Validation runtime
Fitting model: KNeighborsUnif ...
	0.72	 = Validation accuracy score
	0.02s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.744	 = Validation accuracy score
	0.01s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.822	 = Validation accuracy score
	0.82s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.824	 = Validation accuracy score
	1.03s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.802	 = Validation accuracy score
	0.72s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.804	 = Validation accuracy score
	0.72s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: LightGBM ...
	0.832	 = Validation accuracy score
	3.4s	 = Training runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.83

Fitting model: WeightedEnsemble_L2 ...
	0.854	 = Validation accuracy score
	0.35s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 65.82s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("ag_ec2021_demo/")


We can visualize the performance of each model with `predictor.leaderboard()`. Internally, AutoGluon trains a diverse set of different tabular models and computes a weighted ensemble to combine these models.

In [6]:
predictor.leaderboard()

                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2      0.854       1.054986  43.385918                0.000447           0.353042            2       True         14
1         LightGBMLarge      0.834       0.020090   7.005562                0.020090           7.005562            1       True         13
2              LightGBM      0.832       0.040622   3.404332                0.040622           3.404332            1       True          9
3            LightGBMXT      0.830       0.028808   1.973339                0.028808           1.973339            1       True         10
4      RandomForestEntr      0.824       0.102591   1.027688                0.102591           1.027688            1       True          6
5      RandomForestGini      0.822       0.102530   0.824320                0.102530           0.824320            1       True          5
6               XGBoost    

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.854,1.054986,43.385918,0.000447,0.353042,2,True,14
1,LightGBMLarge,0.834,0.020090,7.005562,0.020090,7.005562,1,True,13
2,LightGBM,0.832,0.040622,3.404332,0.040622,3.404332,1,True,9
3,LightGBMXT,0.830,0.028808,1.973339,0.028808,1.973339,1,True,10
4,RandomForestEntr,0.824,0.102591,1.027688,0.102591,1.027688,1,True,6
5,RandomForestGini,0.822,0.102530,0.824320,0.102530,0.824320,1,True,5
6,XGBoost,0.816,0.010418,6.679732,0.010418,6.679732,1,True,12
7,CatBoost,0.812,0.003853,4.765288,0.003853,4.765288,1,True,11
8,ExtraTreesEntr,0.804,0.102545,0.718240,0.102545,0.718240,1,True,8
9,ExtraTreesGini,0.802,0.102536,0.716514,0.102536,0.716514,1,True,7


### Evaluation and Prediction

We can also evaluate the model performance on the heldout predictor dataset by calling `.evaluate()`.

In [7]:
predictor.evaluate(dev_df)

Predictive performance on given data: accuracy = 0.8184


0.8184

To get the prediction, you may just use  `predictor.predict()`.

In [8]:
predictions = predictor.predict(dev_df)
predictions

6084     3
927      5
10919    3
8867     2
14455    7
        ..
6618     5
9591     7
14307    1
1553     1
3        2
Name: Cover_Type, Length: 1250, dtype: int64

For classification problems, we can also use `.predict_proba` to get the probability.

In [9]:
probs = predictor.predict_proba(dev_df)
probs.head(5)

,1,2,3,4,5,6,7
6084,0.000142,0.001430,0.619037,0.332179,0.000683,0.046162,0.000368
927,0.096049,0.418703,0.004163,0.002901,0.459582,0.008953,0.009649
10919,0.023226,0.093982,0.643261,0.000548,0.156072,0.079185,0.003726
8867,0.263310,0.569940,0.009541,0.000104,0.139799,0.002734,0.014572
14455,0.004969,0.003474,0.000023,0.000017,0.000043,0.000016,0.991459


### Load the Predictor

Loading a AutoGluon model is straight-forward. We can directly call `.load()`

In [10]:
predictor_loaded = TabularPredictor.load('ag_ec2021_demo')
predictor_loaded.evaluate(dev_df)

Predictive performance on given data: accuracy = 0.8184


0.8184

### Feature Importance

AutoGluon offers a built-in method for calculating the relative importance of each feature based on [permutation-shuffling](https://scikit-learn.org/stable/modules/permutation_importance.html). In the following, we calculate the feature importance and print the top-10 important features. Here, `importance` means the importance score and the other values give you an understanding of the statistical significance of the calculated score.

In [11]:
importance = predictor.feature_importance(dev_df)
importance.head(10)

Computing feature importance via permutation shuffling for 54 features using 1000 rows with 3 shuffle sets...


	308.54s	= Expected runtime (102.85s per shuffle set)


	40.87s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Elevation,0.403000,0.024576,0.000619,3,0.543826,0.262174
Horizontal_Distance_To_Roadways,0.080000,0.008718,0.001967,3,0.129954,0.030046
Horizontal_Distance_To_Fire_Points,0.059333,0.011240,0.005876,3,0.123739,-0.005072
Horizontal_Distance_To_Hydrology,0.050333,0.004726,0.001463,3,0.077413,0.023254
Wilderness_Area4,0.024000,0.002000,0.001153,3,0.035460,0.012540
Wilderness_Area3,0.018667,0.004619,0.009902,3,0.045133,-0.007800
Hillshade_Noon,0.017667,0.012897,0.070509,3,0.091568,-0.056235
Vertical_Distance_To_Hydrology,0.011667,0.008386,0.068788,3,0.059722,-0.036389
Soil_Type4,0.011000,0.001732,0.004082,3,0.020925,0.001075
Slope,0.008667,0.001528,0.005098,3,0.017420,-0.000086


From the results, we can see that `Elevation` is the most important feature. `Horizontal_Distance_To_Roadways` is the 2nd most important feature.

### Achieve Better Performance

The default behavior of AutoGluon is to compute a weighted ensemble of a diverse set of models. Usually, you can achieve better performance via stack ensembling. To achieve better performance based on automated stack ensembling, you can specify `presets="best_quality"` when calling `.fit()` in AutoGluon. For more details, you can also checkout our provided script. The detailed architecture is described in [1] and we also provide the following figure so you can know the general architecture.

<img src="https://deep-earth.s3.amazonaws.com/datasets/earthcube2021_demo/stacking.png" alt="screenshot" style="width: 500px;"/>

With `.fit(train_df, presets="best_quality")`, we are able to achieve 82/1692 in the competition. To reproduce our number, you may try the command mentioned in [link](https://github.com/sxjscience/EC2021_autogluon_notebook).

<img src="https://deep-earth.s3.amazonaws.com/datasets/earthcube2021_demo/forest_cover_type.png" alt="screenshot" style="width: 500px;"/>

## Solar Radiation Prediction

In the second example, we will train model to predict the solar radiation. The orignal dataset is available in [Kaggle Solar Radiation Prediction](https://www.kaggle.com/dronio/SolarEnergy). The dataset contains such columns as: "wind direction", "wind speed", "humidity" and "temperature". The response parameter that is to be predicted is: "Solar_radiation". It contains measurements for the past 4 months and you have to predict the level of solar radiation. Let's download and load the dataset.

In [12]:
!wget https://deep-earth.s3.amazonaws.com/datasets/earthcube2021_demo/SolarPrediction.csv.zip -O SolarPrediction.csv.zip

--2021-04-14 04:12:01--  https://deep-earth.s3.amazonaws.com/datasets/earthcube2021_demo/SolarPrediction.csv.zip
Resolving deep-earth.s3.amazonaws.com (deep-earth.s3.amazonaws.com)... 52.217.39.180
Connecting to deep-earth.s3.amazonaws.com (deep-earth.s3.amazonaws.com)|52.217.39.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 523425 (511K) [application/zip]
Saving to: ‘SolarPrediction.csv.zip’

SolarPrediction.csv 100%[===================>] 511.16K  --.-KB/s    in 0.007s  

2021-04-14 04:12:01 (74.1 MB/s) - ‘SolarPrediction.csv.zip’ saved [523425/523425]



In [13]:
import pandas as pd
df = pd.read_csv('SolarPrediction.csv.zip')
train_df, dev_df = train_test_split(df, random_state=100)

In [14]:
train_df.head(10)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
2664,1474412104,9/20/2016 12:00:00 AM,12:55:04,1039.15,65,30.40,57,2.26,5.62,06:11:00,18:21:00
12230,1476543319,10/15/2016 12:00:00 AM,04:55:19,1.21,51,30.46,23,181.58,6.75,06:17:00,17:59:00
11706,1476704422,10/17/2016 12:00:00 AM,01:40:22,1.22,50,30.47,39,142.56,10.12,06:18:00,17:58:00
12924,1476330025,10/12/2016 12:00:00 AM,17:40:25,28.35,59,30.45,42,167.42,4.50,06:16:00,18:02:00
27507,1482367563,12/21/2016 12:00:00 AM,14:46:03,637.93,57,30.39,74,40.94,4.50,06:53:00,17:49:00
2516,1474457405,9/21/2016 12:00:00 AM,01:30:05,1.21,45,30.39,73,159.07,3.37,06:11:00,18:20:00
32227,1480723808,12/2/2016 12:00:00 AM,14:10:08,177.19,45,30.34,93,134.78,11.25,06:42:00,17:42:00
12705,1476396922,10/13/2016 12:00:00 AM,12:15:22,1008.08,65,30.46,46,71.24,5.62,06:17:00,18:01:00
14992,1475697322,10/5/2016 12:00:00 AM,09:55:22,292.44,55,30.47,101,18.70,7.87,06:14:00,18:08:00
23615,1478267417,11/4/2016 12:00:00 AM,03:50:17,1.18,44,30.42,38,176.34,7.87,06:25:00,17:47:00


Like in our previos example, we can directly train a predictor with a single `.fit()` call. The difference is that AutoGluon can automatically determine that it is a regression problem.

In [15]:
predictor = TabularPredictor(label='Radiation', eval_metric='r2', path='ag_ec2021_demo2').fit(train_df)

Beginning AutoGluon training ...
AutoGluon will save models to "ag_ec2021_demo2/"
AutoGluon Version:  0.1.0
Train Data Rows:    24514
Train Data Columns: 10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1601.26, 1.11, 206.52072, 315.54334)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    28552.95 MB
	Train Data (Original)  Memory Usage: 7.88 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stag

[1000]	train_set's l2: 5825	train_set's r2: 0.941343	valid_set's l2: 6881.24	valid_set's r2: 0.932405
[2000]	train_set's l2: 4818.35	train_set's r2: 0.951483	valid_set's l2: 6360.95	valid_set's r2: 0.937497
[3000]	train_set's l2: 4202.38	train_set's r2: 0.957684	valid_set's l2: 6212.24	valid_set's r2: 0.938993


	0.9393	 = Validation r2 score
	8.03s	 = Training runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost ...
	0.9423	 = Validation r2 score
	5.65s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: XGBoost ...
	0.9431	 = Validation r2 score
	4.02s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	0.9378	 = Validation r2 score
	99.86s	 = Training runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetFastAI ...


/home/ubuntu/.local/lib/python3.6/site-packages/fastai/callbacks/tracker.py:50: UserWarning: <class 'autogluon.tabular.models.fastainn.callbacks.EarlyStoppingCallbackWithTimeLimit'> conditioned on metric `r2` which is not available. Available metrics are: train_loss, valid_loss, r2_score
  warn(f'{self.__class__} conditioned on metric `{self.monitor}` which is not available. Available metrics are: {", ".join(map(str, self.learn.recorder.names[1:-1]))}')
/home/ubuntu/.local/lib/python3.6/site-packages/fastai/callbacks/tracker.py:50: UserWarning: <class 'autogluon.tabular.models.fastainn.callbacks.SaveModelCallback'> conditioned on metric `r2` which is not available. Available metrics are: train_loss, valid_loss, r2_score
  warn(f'{self.__class__} conditioned on metric `{self.monitor}` which is not available. Available metrics are: {", ".join(map(str, self.learn.recorder.names[1:-1]))}')


		[Errno 2] No such file or directory: '/tmp/tmp08gi6y8t/models/NeuralNetFastAI.pth'
Detailed Traceback:
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.6/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 911, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, **model_fit_kwargs)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 883, in _train_single
    model.fit(X=X, y=y, X_val=X_val, y_val=y_val, **model_fit_kwargs)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/autogluon/core/models/abstract/abstract_model.py", line 405, in fit
    self._fit(**kwargs)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py", line 244, in _fit
    model.load(self.name)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/fastai/basic_train.py", line 269, in load
    state = torch.load(source, map_location=d

We can evaluate on the development set by calling `.evaluate()`. Here, we have specified the model to use [R2 score](https://en.wikipedia.org/wiki/Coefficient_of_determination) so it will report the R2.

In [16]:
predictor.evaluate(dev_df)

Predictive performance on given data: r2 = 0.9544366398127356


0.9544366398127356

Similarly, we can also measure the feature importance.

In [17]:
importance = predictor.feature_importance(dev_df)
importance

Computing feature importance via permutation shuffling for 10 features using 1000 rows with 3 shuffle sets...
	31.34s	= Expected runtime (10.45s per shuffle set)
	7.49s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
UNIXTime,0.976487,0.038612,0.000260,3,1.197739,0.755236
Time,0.095032,0.005177,0.000494,3,0.124695,0.065370
Data,0.043758,0.003209,0.000894,3,0.062149,0.025368
Temperature,0.036498,0.003198,0.001275,3,0.054823,0.018173
TimeSunRise,0.006433,0.004206,0.058923,3,0.030533,-0.017668
Humidity,0.005896,0.000836,0.003315,3,0.010684,0.001108
TimeSunSet,0.003256,0.000726,0.008082,3,0.007416,-0.000903
Pressure,0.001025,0.000768,0.073446,3,0.005426,-0.003375
Speed,-0.000206,0.000696,0.670472,3,0.003782,-0.004195
WindDirection(Degrees),-0.000311,0.001080,0.666522,3,0.005876,-0.006499


## More Information

You may check our website for more information and tutorials: https://auto.gluon.ai/. We also support automatically train models with text, image, and multimodal tabular data.

## References

1. Erickson, Nick and Mueller, Jonas and Shirkov, Alexander and Zhang, Hang and Larroy, Pedro and Li, Mu and Smola, Alexander, AutoGluon-Tabular: Robust and Accurate AutoML for Structured Data, 2020, https://arxiv.org/pdf/2003.06505.pdf